In [131]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib


from astropy.io import fits
import os


from gPhoton import gFind
from gPhoton import gAperture
from gPhoton import gMap
from gPhoton.gphoton_utils import read_lc
import datetime

matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

In [132]:
file = 'III_159A_catalog.dat'
names = ('Star', 'Nobs', 'Smin', 'Smax', 'Smean', 'e_Smean', 'sdS', 'Wmin', 'Wmean', 
         'Date1', 'Date2', 'SName', 'RAdeg DEdeg', 's', 'Bmag', 'Vmag', 'ASCC', 'HIP')
hk = pd.read_table(file, delimiter='|', comment='#', names=names)

radec = hk['RAdeg DEdeg'].str.split(' ', expand=True).loc[:,0:1]
radec.columns = ['RA', 'Dec']
print(radec)

            RA       Dec
0     144.2052  +57.9121
1     176.6498  +50.8832
2     166.3839  +43.5245
3     061.5470  +32.9529
4     025.4714  +02.7044
5     025.4714  +02.7044
6     072.7322  +00.5766
7     184.6508  +23.1201
8     184.7562  +24.8462
9     184.8682  +24.2843
10    185.1899  +25.7659
11    185.4543  +26.5491
12    185.4840  +27.3095
13    185.6032  +22.4642
14    185.6308  +25.8285
15    185.7850  +25.8514
16    185.9243  +26.6015
17    186.2594  +25.5607
18    186.3438  +23.2291
19    186.4665  +26.7767
20    186.7761  +26.8457
21    187.4205  +24.5208
22    263.8923  +61.6828
23    271.3632  +02.5020
24    049.3595  +07.6558
25    049.3595  +07.6558
26    049.3862  +07.6902
27    049.3862  +07.6902
28    052.6264  +20.1034
29    053.2082  +23.6923
...        ...       ...
3081  274.0091  +13.9146
3082  287.2250  +34.0621
3083  287.2250  +34.0621
3084  293.1575  +00.5775
3085  293.1575  +00.5775
3086  295.8551  +10.0893
3087  307.8832  +33.7758
3088  307.8832  +33.7758


In [133]:
# i = 281 # the star w/ 77 visits in GALEX (HD 4174, symbiotic M giant)

i = 261 # 74 visits (HD 3765)



In [134]:
# target = hk['Star'][i]
# ra = float(radec['RA'][i][1:])
# print(ra)
# dec = float(radec['Dec'][i][1:])
# print(dec)

# coordinates that I found
# ra = 10.2920
# dec = 40.2025


# found using isrc data, extremely close to what I found
# ra = 10.20528789
# dec = 40.18717517

step_size = 10. # the time resolution
img_step_size = 30.
img_size = [0.1, 0.1]
phot_rad = 0.0045
# phot_rad = 0.01
ap_in = 0.0060 #changed from .005
ap_out = 0.0070 #changed from .006
# ap_in = 0.015
# ap_out = 0.02



# img_step_size = 30.
#screen saver

print(target)

HD   3765 


In [ ]:
exp_data = gFind(band="NUV", skypos=[ra, dec], exponly=True)
k = 0;
timeArray = []
for tstart, tstop in zip(exp_data['NUV']['t0'], exp_data['NUV']['t1']):
        
        timeArray.append((tstart, tstop))
#         print(tstart, tstop, tstop-tstart)
        print(timeArray[k][0],timeArray[k][1],timeArray[k][1]-timeArray[k][0])
        k = k + 1
#         print(len(timeArray))
#         if k == 5:
#             break
        
            

In [ ]:
(exp_data['NUV']['t0'] - exp_data['NUV']['t0'][0]) / (60.*60.*24.*365.)

In [ ]:
step_size = 100 # seconds

flux = np.zeros(len(exp_data['NUV']['t0']))

aVar = True
counter = 0

currentTime = str(datetime.datetime.now())
currentTime = currentTime.split(" ")
currentTime = currentTime[1].split(":")
currentTime = str(currentTime[0])+":"+str(currentTime[1])

currentDate = str(datetime.date.today()) + "_" + str(counter)
while(aVar):
    if not os.path.exists(currentDate):
        os.makedirs(currentDate)
        os.makedirs(currentDate + "/ds9_Images")
        os.makedirs(currentDate + "/data")
        aVar = False
    else:
        counter = counter + 1 
        currentDate = str(datetime.date.today()) + "_" + str(counter)
        avar = True

print(datetime.datetime.now())
for k in range(len(exp_data['NUV']['t0'])):
    photon_events = gAperture(band='NUV', skypos=[ra, dec], stepsz=step_size, radius= phot_rad,
                              annulus=[ap_in, ap_out], verbose=3, csvfile=currentDate+"/data/"+target+ '_' +str(k)+"_lc.csv",
                              trange=[int(exp_data['NUV']['t0'][k]), int(exp_data['NUV']['t1'][k])+1], 
                              overwrite=True)
    data = read_lc(currentDate+'/data/'+target+ '_' +str(k)+"_lc.csv")
    flux[k] = np.median(data['flux_bgsub'])
    print(datetime.datetime.now(), k)
    
    
    


In [ ]:
plt.figure(figsize=(9,5))
plt.scatter((exp_data['NUV']['t0'] - exp_data['NUV']['t0'][0]) / (60.*60.*24.*365.), flux/1e-14,
           s=50, alpha=0.5, lw=0, color='k')
plt.xlabel('Time (years)')
plt.ylabel('NUV Flux ($10^{-14}$)')
plt.title(target)

plt.savefig(currentDate+'/'+target+'_.png', dpi=150, bbox_inches='tight', pad_inches=0.25)
plt.savefig("Plots/"+currentDate+"_"+currentTime+"_"+target+'_.png', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
#let's explore just around year 6, big scatter
plt.figure(figsize=(9,5))
plt.scatter((exp_data['NUV']['t0'] - exp_data['NUV']['t0'][0]) / (60.*60.*24.*365.), flux/1e-14,
           s=50, alpha=0.5, lw=0, color='k')
plt.xlabel('Time (years)')
plt.ylabel('NUV Flux ($10^{-14}$)')
plt.xlim(5.8,6.2)
plt.title(target)

# plt.savefig(target+'.png', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
exp_data['NUV']['t0'][0]

In [11]:
# lets see if the mean flux for each visit is available? (doubt it...)
# would eliminate need for gAperature step for quick-look evolution!
# exp_data2 = gFind(band="both", skypos=[ra, dec])

In [12]:
# exp_data2
# NOPE....

In [ ]:

for k in range(0,len(timeArray)):  
    w = str(k)
    gMap(band= "NUV", skypos=[10.2920, 40.2025], stepsz=100 , skyrange=[0.010, 0.010],
             cntfile= currentDate+"/ds9_Images/"+"HD_3765_visitNumber_"+w+".fits",
             trange=[timeArray[k][0], timeArray[k][1]])
#   values I found 10.2920hours, 40.2025degrees
#   test with 10.20528789,  40.18717517
#   old skypos=[10.3042, 40.1888]
#   gMap uses degrees for image size
#     if (k ==3):
#         break



In [ ]:
gMap(band='NUV', skypos=[10.3042, 40.1888], skyrange=[0.033, 0.033], cntfile="helix_fuv.fits" )


In [ ]:
exp_data['NUV']

In [8]:

gMap(band= "NUV", skypos=[10.3042, 40.1888], stepsz=100 , skyrange=[0.05, 0.05],
             cntfile= "gj_3765test93_count_movie.fits", trange=[749646829, 749647398])